# Laboratory №2
---
## Multilayer Perceptrons. Deep learning Computation

### student: Karim Safiullin
### group: 4167

Task: 
1. Setup the environment and run all the notebooks  
2. On previously found dataset create multilayer perceptron network for prediction the value by using high-level APIs and perform the following tasks:  
    1. Compare different activations functions (ReLu, Sigmoid, etc.). See how change the result and try to analyse it. 
    1. Try to change size of training/validation data. Plot how change the result of loss function. Analyse from under/overfitting problems. 
    1.  Use regularization. Analyse the changing of the result. 
    1. Perform the dropout method. Analyse the result. 
    1. Perform the Xavier initialization.


In [159]:
import os
import pandas as pd
import numpy as np
%matplotlib inline
import tensorflow as tf
import time
import matplotlib.pyplot as plt
import matplotlib
from sklearn.model_selection import train_test_split

In [65]:
tf.__version__

'2.3.0'

## Reading Dataset  
**Dataset**: Melbourne Housing Market  
**Source**: Kaggle  
**Description**: Melbourne housing clearance data from Jan 2016  

In [114]:
DATASET_PATH = "./data/melb_data.csv"

In [115]:
melbourne_data = pd.read_csv(DATASET_PATH) 

## Column's desciption:
- `Suburb` - name of the suburb **will be used**
- `Adress` - houses location **will not be used**
- `Rooms` - number of rooms in a house **will be used**
- `Type` -  br - bedroom(s); h - house,cottage,villa, semi,terrace; u - unit, duplex; t - townhouse; dev site - development site; o res - other residential. **will be used**
- `Price` - output column for prediction (price of a house) **will be used**
- `Method` - S - property sold; SP - property sold prior; PI - property passed in; PN - sold prior not disclosed; SN - sold not disclosed; NB - no bid; VB - vendor bid; W - withdrawn prior to auction; SA - sold after auction; SS - sold after auction price not disclosed. N/A - price or highest bid not available. **will not be used**
- `SellerG` - name of Real Estate Agent **will not be used**
- `Date` - Date sold **will not be used**
- `Distance` - Distance from CBD (Melbourne city centre) **will be used**
- `Postcode` - postcode of a house same as suburb **will be used**
- `Bedroom2` -  number of bedrooms **will  be used**
- `Bathroom` - amount of bathrooms **will be used**
- `Car` - Number of carspots **will be used**
- `Landsize` -  Land Size **will be used**
- `BuildingArea` - Building Size  **will be used**
- `YearBuilt` - year of a build **will be used**
- `CouncilArea` -  Governing council for the area **will not be used**
- `Lattitude` - latitude of a house **will not be used**
- `Longtitude` - longtitude of a house **will not be used**
- `Regionname` - name of houses region **will be used**
- `Propertycount` -   Number of properties that exist in the suburb. **will be used**

### all strings will be changed at the numbers (for example: `method` column: S - 1, SP - 2 and etc )

In [116]:
melbourne_data.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

In [117]:
melbourne_data.describe()

,Rooms,Price,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
count,13580.000000,1.358000e+04,13580.000000,13580.000000,13580.000000,13580.000000,13518.000000,13580.000000,7130.000000,8205.000000,13580.000000,13580.000000,13580.000000
mean,2.937997,1.075684e+06,10.137776,3105.301915,2.914728,1.534242,1.610075,558.416127,151.967650,1964.684217,-37.809203,144.995216,7454.417378
std,0.955748,6.393107e+05,5.868725,90.676964,0.965921,0.691712,0.962634,3990.669241,541.014538,37.273762,0.079260,0.103916,4378.581772
min,1.000000,8.500000e+04,0.000000,3000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1196.000000,-38.182550,144.431810,249.000000
25%,2.000000,6.500000e+05,6.100000,3044.000000,2.000000,1.000000,1.000000,177.000000,93.000000,1940.000000,-37.856822,144.929600,4380.000000
50%,3.000000,9.030000e+05,9.200000,3084.000000,3.000000,1.000000,2.000000,440.000000,126.000000,1970.000000,-37.802355,145.000100,6555.000000
75%,3.000000,1.330000e+06,13.000000,3148.000000,3.000000,2.000000,2.000000,651.000000,174.000000,1999.000000,-37.756400,145.058305,10331.000000
max,10.000000,9.000000e+06,48.100000,3977.000000,20.000000,8.000000,10.000000,433014.000000,44515.000000,2018.000000,-37.408530,145.526350,21650.000000


In [118]:
melbourne_data.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [119]:
melbourne_data.dtypes

Suburb            object
Address           object
Rooms              int64
Type              object
Price            float64
Method            object
SellerG           object
Date              object
Distance         float64
Postcode         float64
Bedroom2         float64
Bathroom         float64
Car              float64
Landsize         float64
BuildingArea     float64
YearBuilt        float64
CouncilArea       object
Lattitude        float64
Longtitude       float64
Regionname        object
Propertycount    float64
dtype: object

In [120]:
type(melbourne_data)

pandas.core.frame.DataFrame

In [121]:
num_words=melbourne_data['Suburb'].unique().size

In [122]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(melbourne_data['Suburb'])
name1_tokenized = tokenizer.texts_to_sequences(melbourne_data['Suburb'])
name1_tokenized_pad = tf.keras.preprocessing.sequence.pad_sequences(name1_tokenized, maxlen=1, truncating='pre')

In [123]:
name1_tokenized_pad

array([[88],
       [88],
       [88],
       ...,
       [49],
       [49],
       [32]], dtype=int32)

In [124]:
melbourne_data=melbourne_data.assign(Suburb=pd.DataFrame(name1_tokenized_pad, columns=['Suburb']))

In [125]:
num_words=melbourne_data['Regionname'].unique().size

In [126]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(melbourne_data['Regionname'])
name1_tokenized = tokenizer.texts_to_sequences(melbourne_data['Regionname'])
name1_tokenized_pad = tf.keras.preprocessing.sequence.pad_sequences(name1_tokenized, maxlen=1, truncating='pre')

In [127]:
melbourne_data=melbourne_data.assign(Regionname=pd.DataFrame(name1_tokenized_pad, columns=['Regionname']))

In [128]:
melbourne_data = melbourne_data.sample(frac = 1) 

In [129]:
melbourne_data = melbourne_data.dropna(axis=0,subset=['Price'])

In [130]:
melbourne_data.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
2735,20,8 Dudley St,3,h,767500.0,S,Sweeney,28/05/2016,6.4,3011.0,...,2.0,0.0,458.0,98.0,1930.0,Maribyrnong,-37.79420,144.88320,1,7570.0
1680,2,967 Drummond St,4,h,2718000.0,S,Ray,11/03/2017,3.2,3054.0,...,2.0,0.0,538.0,142.0,2015.0,Yarra,-37.78160,144.97140,1,3106.0
60,4,174 Parer Rd,2,h,675000.0,S,Barry,16/07/2016,13.5,3042.0,...,1.0,1.0,612.0,100.0,1975.0,Moonee Valley,-37.71870,144.87650,1,3464.0
13486,29,17 Canadian Ct,3,h,535000.0,SP,Barry,26/08/2017,17.4,3048.0,...,1.0,4.0,572.0,105.0,1980.0,NaN,-37.65905,144.92347,1,4704.0
12977,20,28 Central Av,2,h,942000.0,SP,Jas,19/08/2017,5.1,3011.0,...,1.0,1.0,241.0,91.0,1915.0,NaN,-37.79583,144.89406,1,7570.0


In [131]:
melbourne_data.dtypes

Suburb             int32
Address           object
Rooms              int64
Type              object
Price            float64
Method            object
SellerG           object
Date              object
Distance         float64
Postcode         float64
Bedroom2         float64
Bathroom         float64
Car              float64
Landsize         float64
BuildingArea     float64
YearBuilt        float64
CouncilArea       object
Lattitude        float64
Longtitude       float64
Regionname         int32
Propertycount    float64
dtype: object

In [132]:
melbourne_data.Type

2735     h
1680     h
60       h
13486    h
12977    h
        ..
6769     h
6419     h
5687     h
10605    h
12291    t
Name: Type, Length: 13580, dtype: object

In [134]:
melbourne_data.Type.unique()

array(['h', 't', 'u'], dtype=object)

In [138]:
type_col = melbourne_data["Type"].map({'h':0,'t':1,'u':2})

In [139]:
melbourne_data.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
2735,20,8 Dudley St,3,h,767500.0,S,Sweeney,28/05/2016,6.4,3011.0,...,2.0,0.0,458.0,98.0,1930.0,Maribyrnong,-37.79420,144.88320,1,7570.0
1680,2,967 Drummond St,4,h,2718000.0,S,Ray,11/03/2017,3.2,3054.0,...,2.0,0.0,538.0,142.0,2015.0,Yarra,-37.78160,144.97140,1,3106.0
60,4,174 Parer Rd,2,h,675000.0,S,Barry,16/07/2016,13.5,3042.0,...,1.0,1.0,612.0,100.0,1975.0,Moonee Valley,-37.71870,144.87650,1,3464.0
13486,29,17 Canadian Ct,3,h,535000.0,SP,Barry,26/08/2017,17.4,3048.0,...,1.0,4.0,572.0,105.0,1980.0,NaN,-37.65905,144.92347,1,4704.0
12977,20,28 Central Av,2,h,942000.0,SP,Jas,19/08/2017,5.1,3011.0,...,1.0,1.0,241.0,91.0,1915.0,NaN,-37.79583,144.89406,1,7570.0


In [141]:
melbourne_data=melbourne_data.assign(Type=type_col)

In [142]:
melbourne_data.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
2735,20,8 Dudley St,3,0,767500.0,S,Sweeney,28/05/2016,6.4,3011.0,...,2.0,0.0,458.0,98.0,1930.0,Maribyrnong,-37.79420,144.88320,1,7570.0
1680,2,967 Drummond St,4,0,2718000.0,S,Ray,11/03/2017,3.2,3054.0,...,2.0,0.0,538.0,142.0,2015.0,Yarra,-37.78160,144.97140,1,3106.0
60,4,174 Parer Rd,2,0,675000.0,S,Barry,16/07/2016,13.5,3042.0,...,1.0,1.0,612.0,100.0,1975.0,Moonee Valley,-37.71870,144.87650,1,3464.0
13486,29,17 Canadian Ct,3,0,535000.0,SP,Barry,26/08/2017,17.4,3048.0,...,1.0,4.0,572.0,105.0,1980.0,NaN,-37.65905,144.92347,1,4704.0
12977,20,28 Central Av,2,0,942000.0,SP,Jas,19/08/2017,5.1,3011.0,...,1.0,1.0,241.0,91.0,1915.0,NaN,-37.79583,144.89406,1,7570.0


In [146]:
melbourne_data = melbourne_data.drop(columns=['Address','Method','SellerG','Date','Postcode','CouncilArea','Lattitude','Longtitude'])

In [147]:
melbourne_data.head()

,Suburb,Rooms,Type,Price,Distance,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Regionname,Propertycount
2735,20,3,0,767500.0,6.4,3.0,2.0,0.0,458.0,98.0,1930.0,1,7570.0
1680,2,4,0,2718000.0,3.2,4.0,2.0,0.0,538.0,142.0,2015.0,1,3106.0
60,4,2,0,675000.0,13.5,2.0,1.0,1.0,612.0,100.0,1975.0,1,3464.0
13486,29,3,0,535000.0,17.4,3.0,1.0,4.0,572.0,105.0,1980.0,1,4704.0
12977,20,2,0,942000.0,5.1,2.0,1.0,1.0,241.0,91.0,1915.0,1,7570.0


In [148]:
target = melbourne_data.pop('Price')

In [171]:
train = melbourne_data.sample(frac=0.8,random_state=200)
test = melbourne_data.drop(train.index)

In [ ]:
class MultiPerceptron(tf.keras.Model):

    def __init__(self):
        super(MultiPerceptron, self).__init__()
        self.dense1 = tf.keras.layers.Dense(4, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(5, activation=tf.nn.softmax)
        self.dropout = tf.keras.layers.Dropout(0.5)

    def call(self, inputs, training=False):
        x = self.dense1(inputs)
        if training:
          x = self.dropout(x, training=training)
        return self.dense2(x)

model = MultiPerceptron()